In [1]:
import copy
import glob
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.metrics import classification_report, f1_score

from wrapper import OASIS
from split import split_data

scans_home = 'data/scans'
labels_file = 'data/OASIS3_MRID2Label_052918.csv'
stats_filepath = 'test_downsampled_resnet_outputs.txt'
n_classes = 3
freeze_layers = False
start_freeze_layer = 'Mixed_5d'
use_parallel = True
vision_model = torchvision.models.resnet50()

loss_weights = torch.tensor([1.,1.,5.])
if torch.cuda.is_available():
    loss_weights = loss_weights.cuda()
criterion = nn.CrossEntropyLoss(weight=loss_weights)
optimizer_type = torch.optim.Adam
lr_scheduler_type = optim.lr_scheduler.StepLR
num_epochs = 20
best_model_filepath = None
load_model_filepath = None
#load_model_filepath = 'model_best.pth.tar'

def get_counts(filename_labels):
    counts = [0]*3
    for filename, label in filename_labels:
        counts[label] += 1
    return counts


def train_model(model, dataloaders, datasets, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_f1_score = 0.0
    best_acc = 0.0
    
    # list of models from all epochs
    model_list = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                    model = model.cuda()
                else:
                    input = Variable(inputs)
                    labels = Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = int(running_corrects) / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            with open(stats_filepath, 'a') as f:
                f.write('Epoch {} {} Loss: {:.4f} Acc: {:.4f}\n'.format(epoch, phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val':
                predictions = evaluate_model(model, dataloaders['val'], dataset_sizes['val'], use_cuda)
                true_y = [y for img, y in datasets['val']]
                f1 = f1_score(true_y, predictions, average = 'macro')
                all_f1s = f1_score(true_y, predictions, average = None)
                
                report = classification_report(true_y, predictions)                   
                
                # print f1 score and write to file
                print('macro f1_score: {:.4f}'.format(f1))
                print('all f1_scores: {}'.format(str(all_f1s)))
                print(report)
                with open(stats_filepath, 'a') as f:
                    f.write('Epoch {} macro f1_score = {:.4f} \n'.format(epoch, f1))
                    f.write('all f1_scores: {} \n'.format(str(all_f1s)))
                    f.write('\n Epoch {} \n {} \n\n'.format(epoch, report))
                
                #update epoch acc
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    
                # update best model based on f1_score
                if f1 > best_f1_score:
                    best_f1_score = f1
                    best_model_wts = model.state_dict()

                    state = {'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
                    if best_model_filepath is not None:
                        torch.save(state, best_model_filepath)
        
        model_list.append(copy.deepcopy(model))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    with open(stats_filepath, 'a') as f:
        f.write('Best val Acc: {:4f}\n'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model_list, model


def evaluate_model(model, testset_loader, test_size, use_gpu):
    model.train(False)  # Set model to evaluate mode

    predictions = []
    # Iterate over data
    for inputs, labels in tqdm(testset_loader):
        # TODO: wrap them in Variable?
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()

        # forward
        outputs = model(inputs)
        if type(outputs) == tuple:
            outputs, _ = outputs
        _, preds = torch.max(outputs.data, 1)
        predictions.extend(preds.tolist())
    return predictions


def load_saved_model(filepath, model, optimizer=None):
    state = torch.load(filepath)
    model.load_state_dict(state['state_dict'])
    # Only need to load optimizer if you are going to resume training on the model
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_filenames, val_filenames, test_filenames = split_data(scans_home, labels_file, balanced = True)
print('train filenames size: ', len(train_filenames))
print('validation filenames size: ', len(val_filenames))
print('test filenames size: ', len(test_filenames))
print('label counts for training set: ', get_counts(train_filenames))
print('label counts for validation set: ', get_counts(val_filenames))
print('label counts for test set: ', get_counts(test_filenames))

train_dataset = OASIS(train_filenames, input_size = 224)
val_dataset = OASIS(val_filenames, input_size = 224)
test_dataset = OASIS(test_filenames, input_size = 224)
# print([y for img, y in train_dataset])
# print([y for img, y in val_dataset])
# print([y for img, y in test_dataset])

#print out a sample image shape
'''image_array, label = train_dataset[4]
print(image_array.shape)'''
print('training dataset size: ', len(train_dataset))
print('validation dataset size: ', len(val_dataset))
print('test dataset size: ', len(test_dataset))

# utilize the distribution of classes to probabilistically sample
# batch_size = 20
# class_sample_count = np.array([1000, 500, 250]) #distribution of the class examples
# weights = 1./ class_sample_count
# sample_weights = np.array([weights[t] for img, t in train_dataset])
# sample_weights = torch.from_numpy(sample_weights).double()
# sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights, len(sample_weights))

trainset_loader = DataLoader(train_dataset, batch_size=20, shuffle=False, num_workers=8)
valset_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=8)
testset_loader = DataLoader(test_dataset, batch_size=20, shuffle=False, num_workers=8)

# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# Since imagenet has 1000 classes, we need to change our last layer according to the number of classes we have
n_features = vision_model.fc.in_features
vision_model.fc = nn.Linear(n_features, n_classes)

# Freeze layers if freeze_layer is True
for i, param in vision_model.named_parameters():
    if freeze_layers:
        param.requires_grad = False
    else:
        param.requires_grad = True
if freeze_layers:
    ct = []
    for name, child in vision_model.named_children():
        #if name == 'fc':
        if start_freeze_layer in ct:
            for params in child.parameters():
                params.requires_grad = True
        ct.append(name)
        
# He initialization
def init_weights(m):
    # if type(m) == nn.Linear or type(m) == nn.Conv1d:
    if m.requires_grad:
        nn.init.kaiming_normal_(m.weight)

# To view which layers are freezed and which layers are not freezed:
for name, child in vision_model.named_children():
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)

if use_parallel:
    print("[Using all the available GPUs]")
    vision_model = nn.DataParallel(vision_model, device_ids=[0, 1])

dataloaders = {'train': trainset_loader, 'val': valset_loader}
datasets = {'train': train_dataset, 'val': val_dataset}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
optimizable_params = [param for param in vision_model.parameters() if param.requires_grad]
optimizer = optimizer_type(optimizable_params, lr=0.001)
exp_lr_scheduler = lr_scheduler_type(optimizer, step_size=7, gamma=0.1)

# If we want to load a model with saved parameters
if load_model_filepath is not None:
    load_saved_model(load_model_filepath, vision_model, optimizer)

num labels is 2107
num filenames is 2193
num experiments is 1950
counts per class: [1536, 322, 92]
train filenames size:  515
validation filenames size:  110
test filenames size:  111
label counts for training set:  [225, 225, 65]
label counts for validation set:  [48, 48, 14]
label counts for test set:  [49, 49, 13]


/home/shared/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training dataset size:  7697
validation dataset size:  1650
test dataset size:  1665
cuda
weight True
weight True
bias True
0.conv1.weight True
0.bn1.weight True
0.bn1.bias True
0.conv2.weight True
0.bn2.weight True
0.bn2.bias True
0.conv3.weight True
0.bn3.weight True
0.bn3.bias True
0.downsample.0.weight True
0.downsample.1.weight True
0.downsample.1.bias True
1.conv1.weight True
1.bn1.weight True
1.bn1.bias True
1.conv2.weight True
1.bn2.weight True
1.bn2.bias True
1.conv3.weight True
1.bn3.weight True
1.bn3.bias True
2.conv1.weight True
2.bn1.weight True
2.bn1.bias True
2.conv2.weight True
2.bn2.weight True
2.bn2.bias True
2.conv3.weight True
2.bn3.weight True
2.bn3.bias True
0.conv1.weight True
0.bn1.weight True
0.bn1.bias True
0.conv2.weight True
0.bn2.weight True
0.bn2.bias True
0.conv3.weight True
0.bn3.weight True
0.bn3.bias True
0.downsample.0.weight True
0.downsample.1.weight True
0.downsample.1.bias True
1.conv1.weight True
1.bn1.weight True
1.bn1.bias True
1.conv2.weight T

In [3]:
model_list, best_model = train_model(vision_model,
                             dataloaders,
                             datasets,
                             dataset_sizes,
                             criterion,
                             optimizer,
                             exp_lr_scheduler,
                             use_cuda,
                             num_epochs)

  0%|          | 0/385 [00:00<?, ?it/s]

Epoch 0/19
----------


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  0%|          | 0/83 [00:00<?, ?it/s]

train Loss: 0.0713 Acc: 0.3915


  0%|          | 0/83 [00:00<?, ?it/s]

...

train Loss: 0.0516 Acc: 0.4564


  0%|          | 0/83 [00:00<?, ?it/s]

val Loss: 0.0498 Acc: 0.4642


100%|██████████| 83/83 [00:09<00:00,  9.17it/s]


macro f1_score: 0.2789
all f1_scores: [0.62348555 0.20360551 0.00947867]
             precision    recall  f1-score   support

          0       0.47      0.93      0.62       720
          1       0.43      0.13      0.20       720
          2       1.00      0.00      0.01       210

avg / total       0.52      0.46      0.36      1650


Training complete in 47m 53s
Best val Acc: 0.470909


In [4]:
predictions = evaluate_model(best_model, testset_loader, len(test_dataset), use_cuda)
true_y = [y for img, y in test_dataset]
best_report = classification_report(true_y, predictions)

with open(stats_filepath, 'a') as f:
    f.write('\n Best report \n {}'.format(best_report))   
    print(best_report)

100%|██████████| 84/84 [00:09<00:00,  9.14it/s]


             precision    recall  f1-score   support

          0       0.48      0.88      0.62       735
          1       0.51      0.22      0.31       735
          2       0.00      0.00      0.00       195

avg / total       0.44      0.49      0.41      1665



In [5]:
print('micro f1', f1_score(true_y, predictions, average = 'micro'))
print('macro f1', f1_score(true_y, predictions, average = 'macro'))

print(best_report)

micro f1 0.4870870870870871
macro f1 0.31036121673003797
             precision    recall  f1-score   support

          0       0.48      0.88      0.62       735
          1       0.51      0.22      0.31       735
          2       0.00      0.00      0.00       195

avg / total       0.44      0.49      0.41      1665



In [6]:
prediction_array = np.array(predictions)
true_y_array = np.array(true_y)
test_acc = np.average(prediction_array == true_y_array)
print("test accuracy is {}".format(test_acc))
print()

import pandas as pd
print("prediction\n", pd.Series(prediction_array).value_counts())
print()
print("true values \n", pd.Series(true_y_array).value_counts())
print()

test accuracy is 0.4870870870870871

prediction
 0    1345
1     317
2       3
dtype: int64

true values 
 1    735
0    735
2    195
dtype: int64



In [7]:
print(len(test_dataset))
print(len(val_dataset))

1665
1650
